In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os


batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# The data, split between train and test sets:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
def ANN_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

def train_model(model, x_train, y_train, x_test, y_test, epochs, save_as="cifarmodel"):
    
    mcp_save = keras.callbacks.ModelCheckpoint(save_as+'.hdf5',
                                                       save_best_only=True, 
                                                       monitor='val_loss', 
                                                       mode='min')
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True, 
                  callbacks=[mcp_save])
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None)#,
            # fraction of images reserved for validation (strictly between 0 and 1)
            #validation_split=0.0)

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        history = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            epochs=epochs,
                            #validation_data=(x_test, y_test),
                            workers=4)#,
                            #callbacks=[mcp_save])

        # # Save model and weights
        # if not os.path.isdir(save_dir):
        #     os.makedirs(save_dir)
        # model_path = os.path.join(save_dir, model_name)
        # model.save(model_path)
        # print('Saved trained model at %s ' % model_path)
        
        # Load best model version from training
        model = keras.models.load_model(save_as+'.hdf5')
        # Score trained model.
        scores = model.evaluate(x_test, y_test, verbose=1)
        print('Test loss:', scores[0])
        print('Test accuracy:', scores[1])
        return history.history, scores


In [ ]:
model = ANN_model()
hist, scores = train_model(model, x_train, y_train, x_test, y_test,epochs=2)

In [4]:
from fedml import Alliance, AllianceMember,KerasSequentialBaseLearner, FedAveragingClassifier
from read_data import read_training_data
import numpy as np

alliance = Alliance()
M=10
def split_data(x,y,fractions):
    s = len(x)
    new_order = np.random.permutation(s)
    x = x[new_order]
    y = y[new_order]
    if type(fractions) is int:
        fraction_splits = np.int32(np.linspace(0,s,fractions+1))
        nr = fractions
    elif type(fractions)==np.ndarray or type(fractions)==list:
        parts = np.array(fractions)*s/np.sum(fractions)
        fraction_splits = np.int32([np.sum(parts[:i]) for i in range(len(parts)+1)])
        nr = len(parts)
        
    x_list = []
    y_list = []
    for i in range(nr):
        x_list.append(x[fraction_splits[i]:fraction_splits[i+1]]) 
        y_list.append(y[fraction_splits[i]:fraction_splits[i+1]])

    return x_list,y_list
F = [10,10,10,5,4,1]
M = len(F)
x_train_splits, y_train_splits = split_data(x_train,y_train,fractions=F)
x_test_splits, y_test_splits = split_data(x_test,y_test,fractions=F)
print("x_train shape: ", np.array(x_train).shape)
tot_len = 0
for split in x_train_splits:
    print("x_train_splits shape: ", split.shape)
    tot_len += len(split)
print("tot len: ", tot_len)
classes = np.arange(10)
for i in range(M):
    member = AllianceMember(x_train_splits[i], y_train_splits[i], classes=classes)
    alliance.add_member(member)

x_train shape:  (50000, 32, 32, 3)
x_train_splits shape:  (12500, 32, 32, 3)
x_train_splits shape:  (12500, 32, 32, 3)
x_train_splits shape:  (12500, 32, 32, 3)
x_train_splits shape:  (6250, 32, 32, 3)
x_train_splits shape:  (5000, 32, 32, 3)
x_train_splits shape:  (1250, 32, 32, 3)
tot len:  50000


In [5]:
from cifarmodel import KerasSequentialCifar

base_learner = KerasSequentialCifar()
fedavg_model = FedAveragingClassifier(alliance=alliance,base_learner=base_learner)


In [ ]:
parameters = {"nr_global_iterations": 100, "nr_local_iterations":1} 
training_loss, test_loss = fedavg_model.fit(parameters)

global epoch:  0
virtual memory used:  82.0 %
inside partial fit data order shape:  (12500,)
data order shape:  (12500,)
initiate datagen
ind shape:  (3200,) , ind max:  3199 , x shape:  (12500, 32, 32, 3)
Epoch 1/1
100/100 [==============================] - 13s 127ms/step - loss: 2.5713 - accuracy: 0.1553
partial fit ends
outside partial mode
updated attributes
inside partial fit data order shape:  (5000,)
data order shape:  (5000,)
initiate datagen
ind shape:  (3200,) , ind max:  3199 , x shape:  (5000, 32, 32, 3)
Epoch 1/1
100/100 [==============================] - 13s 129ms/step - loss: 2.5424 - accuracy: 0.1553
partial fit ends
outside partial mode
updated attributes
inside partial fit data order shape:  (12500,)
data order shape:  (12500,)
initiate datagen
ind shape:  (3200,) , ind max:  3199 , x shape:  (12500, 32, 32, 3)
Epoch 1/1
100/100 [==============================] - 13s 134ms/step - loss: 2.5840 - accuracy: 0.1603
partial fit ends
outside partial mode
updated attributes


100/100 [==============================] - 11s 114ms/step - loss: 2.3145 - accuracy: 0.1838
partial fit ends
outside partial mode
updated attributes
inside partial fit data order shape:  (5000,)
data order shape:  (5000,)
ind shape:  (1800,) , ind max:  4999 , x shape:  (5000, 32, 32, 3)
Epoch 1/1
57/57 [==============================] - 6s 106ms/step - loss: 2.3824 - accuracy: 0.1861
partial fit ends
outside partial mode
updated attributes
mean avg:  0.038056243
12500/12500 [==============================] - 7s 554us/step
errRate:  1.57536 validation:  [2.0751389129257203, 0.2123199999332428]
12500/12500 [==============================] - 8s 667us/step
errRate:  1.58288 validation:  [2.1151248049163818, 0.2085600048303604]
12500/12500 [==============================] - 7s 600us/step
errRate:  1.55008 validation:  [2.095642643356323, 0.22495999932289124]
12500/12500 [==============================] - 7s 548us/step
errRate:  1.64576 validation:  [2.1984153052520754, 0.17712000012397766]

12500/12500 [==============================] - 7s 536us/step
errRate:  1.41504 validation:  [2.008547258453369, 0.2924799919128418]
12500/12500 [==============================] - 8s 645us/step
errRate:  1.44704 validation:  [1.9552607330703735, 0.2764799892902374]
12500/12500 [==============================] - 8s 621us/step
errRate:  1.44896 validation:  [1.990734442138672, 0.275519996881485]
12500/12500 [==============================] - 10s 831us/step
errRate:  1.49184 validation:  [2.0124862759780884, 0.25407999753952026]
12500/12500 [==============================] - 8s 662us/step
errRate:  1.68848 validation:  [2.391551947555542, 0.1557600051164627]
12500/12500 [==============================] - 8s 635us/step
errRate:  1.42 validation:  [2.0130298336791994, 0.28999999165534973]
12500/12500 [==============================] - 8s 654us/step
errRate:  1.45024 validation:  [1.9569477323913573, 0.27487999200820923]
12500/12500 [==============================] - 8s 625us/step
errRate:  1

6250/6250 [==============================] - 4s 706us/step
errRate:  1.54752 validation:  [2.058931547622681, 0.22623999416828156]
6250/6250 [==============================] - 3s 535us/step
errRate:  1.61088 validation:  [2.1575698457336427, 0.1945600062608719]
6250/6250 [==============================] - 4s 574us/step
errRate:  1.52192 validation:  [2.0280925105285643, 0.23904000222682953]
5000/5000 [==============================] - 3s 572us/step
errRate:  1.2988 validation:  [1.7785413661956788, 0.350600004196167]
5000/5000 [==============================] - 3s 603us/step
errRate:  1.6504 validation:  [2.4664366565704348, 0.17479999363422394]
5000/5000 [==============================] - 4s 729us/step
errRate:  1.524 validation:  [2.055782118988037, 0.23800000548362732]
5000/5000 [==============================] - 3s 547us/step
errRate:  1.3992 validation:  [1.864513097190857, 0.3003999888896942]
5000/5000 [==============================] - 3s 589us/step
errRate:  1.5228 validation: 

1250/1250 [==============================] - 1s 554us/step
errRate:  1.4432 validation:  [1.9094834491729735, 0.2784000039100647]
1250/1250 [==============================] - 1s 555us/step
errRate:  1.2352 validation:  [1.7438285064697265, 0.3824000060558319]
1250/1250 [==============================] - 1s 562us/step
errRate:  1.4192 validation:  [1.9692560899734497, 0.2903999984264374]
member global score:  [0.21 0.24 0.26 0.35 0.38]
member global score:  [0.21 0.22 0.15 0.18 0.38]
member global score:  [0.17 0.22 0.11 0.23 0.28]
member global score:  [0.21 0.23 0.29 0.29 0.37]
member global score:  [0.2  0.18 0.28 0.2  0.29]
member global score:  [0.17 0.2  0.28 0.23 0.23]
12500/12500 [==============================] - 6s 517us/step
errRate:  1.26848 validation:  [1.771200845413208, 0.3657599985599518]
12500/12500 [==============================] - 8s 667us/step
errRate:  1.27152 validation:  [1.7791071536254883, 0.36423999071121216]
12500/12500 [==============================] - 7s 

...

12500/12500 [==============================] - 7s 528us/step
errRate:  1.17776 validation:  [1.6289393141937256, 0.4111199975013733]
12500/12500 [==============================] - 7s 524us/step
errRate:  1.17104 validation:  [1.5633517169189453, 0.4144800007343292]
12500/12500 [==============================] - 7s 532us/step
errRate:  1.0456 validation:  [1.4435046928787232, 0.4772000014781952]
12500/12500 [==============================] - 7s 523us/step
errRate:  1.09152 validation:  [1.4871122513198853, 0.45423999428749084]
12500/12500 [==============================] - 6s 515us/step
errRate:  1.18352 validation:  [1.6181440737915038, 0.4082399904727936]
12500/12500 [==============================] - 7s 528us/step
errRate:  1.16416 validation:  [1.614390497970581, 0.41791999340057373]
6250/6250 [==============================] - 3s 518us/step
errRate:  1.16576 validation:  [1.5603449629974364, 0.4171200096607208]
6250/6250 [==============================] - 3s 514us/step
errRate:  1.

6250/6250 [==============================] - 3s 510us/step
errRate:  1.2 validation:  [1.641060168685913, 0.4000000059604645]
6250/6250 [==============================] - 3s 528us/step
errRate:  1.03552 validation:  [1.4166936628723144, 0.4822399914264679]
6250/6250 [==============================] - 3s 524us/step
errRate:  1.11648 validation:  [1.5461228261566162, 0.44176000356674194]
5000/5000 [==============================] - 3s 528us/step
errRate:  1.0696 validation:  [1.4936929080963135, 0.4652000069618225]
5000/5000 [==============================] - 3s 509us/step
errRate:  1.0976 validation:  [1.5088309867858887, 0.451200008392334]
5000/5000 [==============================] - 3s 510us/step
errRate:  1.1952 validation:  [1.6201798366546631, 0.4023999869823456]
5000/5000 [==============================] - 3s 537us/step
errRate:  1.0888 validation:  [1.5244954677581788, 0.45559999346733093]
5000/5000 [==============================] - 3s 534us/step
errRate:  1.1096 validation:  [1

1250/1250 [==============================] - 1s 531us/step
errRate:  1.1664 validation:  [1.589951724243164, 0.41679999232292175]
1250/1250 [==============================] - 1s 549us/step
errRate:  1.0144 validation:  [1.392661809539795, 0.4927999973297119]
1250/1250 [==============================] - 1s 519us/step
errRate:  1.0208 validation:  [1.4338295532226562, 0.4896000027656555]
1250/1250 [==============================] - 1s 519us/step
errRate:  1.0304 validation:  [1.434553791809082, 0.4848000109195709]
1250/1250 [==============================] - 1s 499us/step
errRate:  1.1104 validation:  [1.4825782478332519, 0.4447999894618988]
member global score:  [0.21 0.24 0.26 0.35 0.38 0.41 0.36 0.43 0.38 0.42 0.46 0.42]
member global score:  [0.21 0.22 0.15 0.18 0.38 0.25 0.39 0.38 0.4  0.48 0.44 0.49]
member global score:  [0.17 0.22 0.11 0.23 0.28 0.41 0.4  0.33 0.43 0.5  0.4  0.48]
member global score:  [0.21 0.23 0.29 0.29 0.37 0.3  0.28 0.35 0.41 0.46 0.44 0.48]
member global sc

12500/12500 [==============================] - 6s 517us/step
errRate:  0.94352 validation:  [1.2994976267623901, 0.5282400250434875]
12500/12500 [==============================] - 7s 527us/step
errRate:  0.94912 validation:  [1.306418332824707, 0.525439977645874]
12500/12500 [==============================] - 7s 546us/step
errRate:  0.94624 validation:  [1.3019011666107179, 0.5268800258636475]
6250/6250 [==============================] - 3s 553us/step
errRate:  0.93376 validation:  [1.2867683742523193, 0.5331199765205383]
5000/5000 [==============================] - 3s 512us/step
errRate:  0.9124 validation:  [1.2773945110321046, 0.5437999963760376]
1250/1250 [==============================] - 1s 498us/step
errRate:  0.9504 validation:  [1.2774339027404784, 0.5248000025749207]
training loss:  [0.1  0.17 0.23 0.34 0.37 0.43 0.46 0.46 0.47 0.5  0.5  0.52 0.53]
global epoch:  13
virtual memory used:  83.2 %
inside partial fit data order shape:  (12500,)
data order shape:  (12500,)
ind sha

100/100 [==============================] - 10s 103ms/step - loss: 1.5075 - accuracy: 0.4588
partial fit ends
outside partial mode
updated attributes
inside partial fit data order shape:  (1250,)
data order shape:  (1250,)
ind shape:  (600,) , ind max:  1246 , x shape:  (1250, 32, 32, 3)
Epoch 1/1
19/19 [==============================] - 2s 106ms/step - loss: 1.4300 - accuracy: 0.4917
partial fit ends
outside partial mode
updated attributes
inside partial fit data order shape:  (12500,)
data order shape:  (12500,)
ind shape:  (3200,) , ind max:  12498 , x shape:  (12500, 32, 32, 3)
Epoch 1/1
100/100 [==============================] - 10s 101ms/step - loss: 1.4915 - accuracy: 0.4572
partial fit ends
outside partial mode
updated attributes
inside partial fit data order shape:  (12500,)
data order shape:  (12500,)
ind shape:  (3200,) , ind max:  12499 , x shape:  (12500, 32, 32, 3)
Epoch 1/1
100/100 [==============================] - 10s 104ms/step - loss: 1.5099 - accuracy: 0.4528
partial